In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Activation,Dropout,Dense

/home/phantom/.conda/envs/deep/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train = pd.read_csv('train.csv',index_col = 0)

train.head()


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train = train.drop(['Name','SibSp','Age','Parch','Embarked','Ticket','Fare','Cabin'],axis =1)
train.head()

,Survived,Pclass,Sex
PassengerId,,,
1,0,3,male
2,1,1,female
3,1,3,female
4,1,1,female
5,0,3,male


In [5]:
d={'male':0,'female':1}
train.Sex=train.Sex.map(d)

train.head()

,Survived,Pclass,Sex
PassengerId,,,
1,0,3,0
2,1,1,1
3,1,3,1
4,1,1,1
5,0,3,0


In [6]:
x_train=np.array(train[['Sex','Pclass']])
print(x_train.shape)



(891, 2)


In [7]:
y_train = np.array(train['Survived'].tolist())
y_train = np.reshape(y_train,(len(y_train),1))
print('Output shape of training data is ',y_train.shape)
print(y_train[:10])

Output shape of training data is  (891, 1)
[[0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]]


In [8]:
model = Sequential([
    Dense(20, input_shape=(2,)),
    Activation('relu'),
    Dense(10),
    Activation('relu'),
    Dense(5),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                60        
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 55        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6         
__________

In [9]:
model.fit(x_train, y_train,
          validation_split = 0.2,
          epochs=50,
          batch_size=30,
          )

Train on 712 samples, validate on 179 samples
Epoch 1/50
712/712 [==============================] - 1s 2ms/step - loss: 0.7725 - acc: 0.6096 - val_loss: 0.6511 - val_acc: 0.6425
Epoch 2/50
712/712 [==============================] - 0s 50us/step - loss: 0.6727 - acc: 0.6096 - val_loss: 0.6199 - val_acc: 0.6425
Epoch 3/50
712/712 [==============================] - 0s 58us/step - loss: 0.6449 - acc: 0.6096 - val_loss: 0.5961 - val_acc: 0.6425
Epoch 4/50
712/712 [==============================] - 0s 55us/step - loss: 0.6198 - acc: 0.6096 - val_loss: 0.5708 - val_acc: 0.6425
Epoch 5/50
712/712 [==============================] - 0s 49us/step - loss: 0.5972 - acc: 0.6306 - val_loss: 0.5595 - val_acc: 0.7598
Epoch 6/50
712/712 [==============================] - 0s 52us/step - loss: 0.5799 - acc: 0.7037 - val_loss: 0.5526 - val_acc: 0.8156
Epoch 7/50
712/712 [==============================] - 0s 58us/step - loss: 0.5678 - acc: 0.7289 - val_loss: 0.5330 - val_acc: 0.8156
Epoch 8/50
712/712 [====

In [10]:
test = pd.read_csv('test.csv',index_col = 0)
test = test.drop(['Name','SibSp','Age','Parch','Embarked','Ticket','Fare','Cabin'],axis =1)
#train.dropna(subset=['Age'], inplace=True)
test.head()

,Pclass,Sex
PassengerId,,
892,3,male
893,3,female
894,2,male
895,3,male
896,3,female


In [11]:
d={'male':0,'female':1}
test.Sex=test.Sex.map(d)

test.head()

,Pclass,Sex
PassengerId,,
892,3,0
893,3,1
894,2,0
895,3,0
896,3,1


In [12]:
x_test=np.array(test[['Sex','Pclass']])
print(x_test.shape)



(418, 2)


In [14]:
y = model.predict(x_test)
y_test = []
for each in y:
    if each<0.5:
        each = 0
    else:
        each = 1
    y_test.append(each)   
pid = []
for i in range(0,len(y_test)):
    pid.append(i+892)
data = {'PassengerId' : pid,
          'Survived' :y_test
          }
result= pd.DataFrame(data)
result.set_index('PassengerId',inplace = True)
result.head(10)

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
897,0
898,1
899,0
900,1


In [14]:
result.to_csv('Result.csv')